<a href="https://colab.research.google.com/github/SaraBatistaPereira/GSI073/blob/main/c_pia_de_gsi073_aula0_seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparação dos dados

Esta tarefa é inverter sequências de caracteres. Exemplo: **aabcd** em **dcbaa**.


In [ ]:
import torch
import torch.nn as nn
import random

# Definindo o vocabulário, note que o original incluía 'e', mas o seu código só tem 'abcd '
chars = list("abcde ") # Usei 'abcde ' para manter a coerência com a resposta anterior
vocab = {ch: i for i, ch in enumerate(chars)} # Cada letra, ganha um número
inv_vocab = {i: ch for ch, i in vocab.items()}# Tabela de decodificação
vocab_size = len(vocab)

def encode(s): # Codifica letras em números
    return torch.tensor([vocab[c] for c in s], dtype=torch.long)

def decode(t): # Decodifica números em letras
    return ''.join(inv_vocab[int(x)] for x in t)

def random_seq(n=5): # Cria novas sequências
    # O 'chars[:-1]' garante que o espaço não seja escolhido como caractere de sequência
    return ''.join(random.choice(chars[:-1]) for _ in range(n))

# Gerar dados
pairs = [(encode(s), encode(s[::-1])) for s in [random_seq() for _ in range(50000)]]

max_len = max(len(x) for x, _ in pairs) # pega maior sequência

def pad(x):  # Preenche conjunto de dados em pad no último índice
    return torch.cat([x, torch.tensor([vocab[' ']] * (max_len - len(x)))], dim=0)

inputs = torch.stack([pad(x) for x, _ in pairs])
targets = torch.stack([pad(y) for _, y in pairs])

train_ds = torch.utils.data.TensorDataset(inputs, targets)
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=128, shuffle=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Linha adicionada para ver um par de dados (solicitado no prompt original)
print(f"Número de sequências geradas: {len(pairs)}")
print(f"Comprimento máximo (max_len): {max_len}")
print("---")
print(f"Exemplo de par (Índice 1):")
print(f"Sequência de Entrada (codificada): {pairs[1][0]}")
print(f"Sequência Alvo (codificada): {pairs[1][1]}")
print(f"Decodificação da Entrada: {decode(pairs[1][0])}")
print(f"Decodificação da Alvo: {decode(pairs[1][1])}")

## Veja um par

In [ ]:
print(pairs[1])

# Definição do modelo Seq2Seq com GRU

In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, emb_size)
        self.gru = nn.GRU(emb_size, hidden_size, batch_first=True)

    def forward(self, x):
        x = self.embed(x)
        # Retorna o estado oculto final [num_layers, B, H]
        _, h = self.gru(x)
        return h

class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, emb_size)
        self.gru = nn.GRU(emb_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, h):
        """
        x: tensor de token(s) de entrada do decoder (para teacher forcing ou geração passo a passo)
        h: estado oculto do encoder/estado oculto prévio do decoder
        """
        x = self.embed(x)
        # out: [B, Seq_Len, H]
        out, h = self.gru(x, h)
        # logits: [B, Seq_Len, Vocab_Size]
        logits = self.fc(out)
        return logits, h

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, tgt):
        # 1. Obtém o estado oculto do encoder (vetor de contexto)
        h = self.encoder(src)
        # 2. Usa o tgt (deslocado) e o vetor de contexto para gerar logits
        # tgt[:, :-1] é usado como entrada do decoder (teacher forcing)
        logits, _ = self.decoder(tgt[:, :-1], h)
        return logits

# Código para usar o modelo treinado: inferência

In [ ]:
def decode_step(decoder, token, h):
    # token: [B, 1] (apenas o último token gerado)
    logits, h = decoder(token, h)
    # next_token: [B, 1] (o token com maior probabilidade)
    next_token = logits[:, -1, :].argmax(-1, keepdim=True)
    return next_token, h

def predict(model, seq, max_len=max_len):
    model.eval()
    with torch.no_grad():
        # Prepara a entrada do encoder
        src = pad(encode(seq)).unsqueeze(0).to(device, dtype=torch.long)
        h = model.encoder(src) # Obtém o estado do encoder

        # Inicializa a geração com o token de padding/start
        token = torch.tensor([[vocab[' ']]], dtype=torch.long, device=device)
        seq_invertida = []
        for _ in range(max_len):
            # Gera o próximo token e atualiza o estado
            token, h = decode_step(model.decoder, token, h)

            # Para a geração se o modelo gerar o token de padding
            if token.item() == vocab[' ']:
                 break

            seq_invertida.append(token.item())

        return decode(seq_invertida)

# Preparação para treino

In [ ]:
# Preparação para treino
emb_size = 32
hidden_size = 64
# 1. Inicializa o Encoder e o Decoder
encoder = Encoder(vocab_size, emb_size, hidden_size)
decoder = Decoder(vocab_size, emb_size, hidden_size)
# 2. Combina no modelo Seq2Seq e move para o dispositivo
model = Seq2Seq(encoder, decoder).to(device)

# 3. Define a função de perda (ignora o padding: " ")
loss_fn = nn.CrossEntropyLoss(ignore_index=vocab[' '])

# 4. Define o otimizador Adam
# AJUSTE CHAVE: A Taxa de Aprendizado (lr) foi reduzida de 1e-3 para 1e-4.
# Isso fará com que o modelo aprenda de forma mais lenta e precisa, ajudando a quebrar a barreira dos 60%.
opt = torch.optim.Adam(model.parameters(), lr=1e-4)

# Execução do treino

In [ ]:
# Execução do treino
print("--- Iniciando Treinamento (10 Épocas) ---")
for epoch in range(25):
    model.train() # Garante que o modelo esteja no modo de treino (ex: ativa Dropout, se houvesse)
    total_loss = 0

    # Itera sobre os lotes (batches) de dados de treinamento
    for xb, yb in train_dl:
        # 1. Preparação: Move o lote para a GPU/CPU
        xb, yb = xb.to(device, dtype=torch.long), yb.to(device, dtype=torch.long)

        # 2. Reinicialização: Zera os gradientes acumulados da etapa anterior
        opt.zero_grad()

        # 3. Forward Pass: O modelo faz a previsão
        # O modelo(xb, yb) executa: Encoder(xb) -> Obtém vetor de contexto 'h' -> Decoder(yb[:, :-1], h) -> Retorna logits
        logits = model(xb, yb)

        # 4. Cálculo da Perda (Loss)
        # Compara a saída prevista (logits) com a saída real esperada (yb[:, 1:])
        # O reshape é necessário para linearizar os dados para a nn.CrossEntropyLoss
        loss = loss_fn(logits.reshape(-1, vocab_size), yb[:, 1:].reshape(-1))

        # 5. Backward Pass: Calcula os gradientes (derivadas da perda em relação aos pesos)
        loss.backward()

        # 6. Otimização: Ajusta os pesos do modelo com base nos gradientes calculados
        opt.step()

        total_loss += loss.item()

    # Imprime a perda média da época
    print(f"Epoch {epoch+1}: loss={total_loss/len(train_dl):.4f}")

In [ ]:
# --- Função de Acurácia (Pode ser definida fora do loop) ---
def calculate_accuracy(pred_seq, target_seq):
    """Calcula a proporção de caracteres corretos por posição."""
    # O target_seq é a string invertida, s[::-1]

    # Compara a string prevista com a string alvo, usando o comprimento da alvo como base
    target_length = len(target_seq)

    # Se a string prevista for mais curta, preenche com um caractere neutro ('-') para comparação
    # Se for mais longa, o excesso é automaticamente penalizado (contado como erro)
    pred_seq_padded = pred_seq.ljust(target_length, '-')

    correct_chars = 0

    # Acurácia de caractere: verifica a posição exata
    for i in range(target_length):
        if i < len(pred_seq) and pred_seq[i] == target_seq[i]:
            correct_chars += 1

    # Retorna o número de caracteres corretos e o comprimento da sequência alvo (total)
    return correct_chars, target_length

# Vamos testar
print("\n--- Teste de Inferência e Cálculo de Acurácia Média ---")

total_correct_chars = 0
total_chars_tested = 0

for i in range(10):
    s = random_seq() # Gera uma nova sequência aleatória
    target = s[::-1] # Sequência alvo correta

    # Obtém a previsão do modelo
    pred = predict(model, s, max_len=len(s))

    # Calcula a proporção de acertos para esta sequência
    correct_count, target_length = calculate_accuracy(pred, target)
    accuracy_ratio = correct_count / target_length
    accuracy_percent = accuracy_ratio * 100

    # Acumula os totais para o cálculo da média geral
    total_correct_chars += correct_count
    total_chars_tested += target_length

    # Classificação geral da sequência
    if pred == target:
        status = "✅ ACERTO TOTAL"
    elif accuracy_ratio > 0:
        status = "⚠️ ACERTO PARCIAL"
    else:
        status = "❌ ERRO TOTAL"

    print(f"Original: {s} -> Alvo: {target}")
    print(f"Previsto: {pred} | {status}")
    print(f"Acurácia Caractere: {correct_count}/{target_length} ({accuracy_percent:.2f}%)")
    print("-" * 40)

# --- Média Geral ---
if total_chars_tested > 0:
    media_geral = (total_correct_chars / total_chars_tested) * 100
    print(f"**ACURÁCIA MÉDIA GERAL nos 10 testes:** {total_correct_chars}/{total_chars_tested} ({media_geral:.2f}%)")
else:
    print("Não foi possível calcular a média geral (zero caracteres testados).")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Vamos testar

In [ ]:
for _ in range(10):
    s = random_seq()
    pred = predict(model, s, max_len=len(s))
    print(f"{s} -> {pred}")


# Exercício
Compare o resultado do uso do encoder de de sequências muito similares e muito diferentes. Por exemplo, codifique "aaaabb", "bbaaab", "cbcaccc" e "cccacbc" e depois faça uma figura das 2 componentes principais usando o método Principal Components Analysis (PCA) do pacote `sklearn.decomposition.PCA`.